# Web-Scraping-Assignment 3

# Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars.

In [11]:
!pip install openpyxl


In [14]:
import openpyxl
from bs4 import BeautifulSoup

def parse_html_and_write_to_excel(html_file, excel_file):
    with open(html_file, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')

    result_info_bars = soup.find_all('div', class_='s-widget-container s-spacing-small s-widget-container-height-small celwidget slot=UPPER template=RESULT_INFO_BAR widgetId=result-info-bar')

    workbook = openpyxl.Workbook()
    worksheet = workbook.active

    worksheet.append(['Product_Name', 'Product_Price', 'Product_Reviews'])

    for result_info_bar in result_info_bars:
        product_name = ""
        product_price = ""
        product_reviews = ""
        
        product_name_tag = result_info_bar.find('span', class_='a-size-medium a-color-base a-text-normal')
        if product_name_tag:
            product_name = product_name_tag.get_text(strip=True)

        product_price_tag = result_info_bar.find('span', class_='a-price-whole')
        if product_price_tag:
            product_price = product_price_tag.get_text(strip=True)
            
        product_reviews_tag = result_info_bar.find('span', class_='a-icon-alt')
        if product_reviews_tag:
            product_reviews = product_reviews_tag.get_text(strip=True)

        worksheet.append([product_name, product_price, product_reviews])

    workbook.save(excel_file)

if __name__ == "__main__":
    html_file = 'Amazon.in.html' 
    excel_file = 'amazon_products.xlsx' 

    parse_html_and_write_to_excel(html_file, excel_file)


FileNotFoundError: [Errno 2] No such file or directory: 'Amazon.in.html'

# In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def scrape_product_details(product):
    details = {
        'Brand Name': '-',
        'Name of the Product': '-',
        'Price': '-',
        'Return/Exchange': '-',
        'Expected Delivery': '-',
        'Availability': '-',
        'Product URL': '-'
    }

    try:
        details['Brand Name'] = product.find('span', class_='a-size-base-plus').get_text(strip=True)
    except AttributeError:
        pass

    try:
        details['Name of the Product'] = product.find('span', class_='a-text-normal').get_text(strip=True)
    except AttributeError:
        pass

    try:
        details['Price'] = product.find('span', class_='a-offscreen').get_text(strip=True)
    except AttributeError:
        pass

    try:
        details['Return/Exchange'] = product.find('div', class_='a-row a-size-base a-color-secondary').get_text(strip=True)
    except AttributeError:
        pass

    try:
        details['Expected Delivery'] = product.find('span', class_='a-text-bold').get_text(strip=True)
    except AttributeError:
        pass

    try:
        details['Availability'] = product.find('span', class_='a-size-medium a-color-success').get_text(strip=True)
    except AttributeError:
        pass

    try:
        details['Product URL'] = 'https://www.amazon.in' + product.find('a', class_='a-link-normal')['href']
    except AttributeError:
        pass

    return details

def search_amazon_and_scrape(product_name, num_pages=3):
    driver = webdriver.Chrome(executable_path='/path/to/chromedriver')

    product_details_list = []
    
if __name__ == '__main__':
    
    try:
        product_name = input('Enter the product you want to search on Amazon.in: ')
        search_amazon_and_scrape(product_name)
    except KeyboardInterrupt:
        print("\nOperation interrupted by the user.")
        
    for page in range(1, num_pages + 1):
        driver.get(f'https://www.amazon.in/s?k={product_name}&page={page}')
        driver.implicitly_wait(10)
        
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        products = soup.find_all('div', class_='s-result-item')

        for product in products:
            details = scrape_product_details(product)
            product_details_list.append(details)

    driver.quit()

    df = pd.DataFrame(product_details_list)

    df.to_csv('amazon_product_details.csv', index=False)

if __name__ == '__main__':
    product_name = input('Enter the product you want to search on Amazon.in: ')
    search_amazon_and_scrape(product_name)



Operation interrupted by the user.


NameError: name 'num_pages' is not defined

In [22]:
!pip install google_images_download

# Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [17]:
!pip install selenium beautifulsoup4
!pip install --upgrade selenium



     ---------------------------------------- 10.2/10.2 MB 9.2 MB/s eta 0:00:00
  Attempting uninstall: selenium
    Found existing installation: selenium 4.15.1
    Uninstalling selenium-4.15.1:
      Successfully uninstalled selenium-4.15.1


In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import requests
import os

def scrape_images(search_keywords, num_images=10):
    driver = webdriver.Chrome(executable_path='path/to/chromedriver')  

    for keyword in search_keywords:
        try:
            search_images(driver, keyword, num_images)
        except Exception as e:
            print(f"An error occurred while searching for '{keyword}': {e}")

    driver.quit()

def search_images(driver, keyword, num_images):
    driver.get("https://images.google.com/")
    time.sleep(2)

    search_box = driver.find_element("name", "q")
    search_box.clear()
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.RETURN)
    time.sleep(2)

    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    img_tags = soup.find_all('img', class_='rg_i')

    image_urls = []
    for img_tag in img_tags[:num_images]:
        img_url = img_tag.get('data-src')
        if img_url:
            image_urls.append(img_url)

    download_images(keyword, image_urls)

def download_images(keyword, image_urls):
    save_folder = f"{keyword}_images"
    os.makedirs(save_folder, exist_ok=True)

    for i, img_url in enumerate(image_urls):
        try:
            response = requests.get(img_url)
            with open(os.path.join(save_folder, f"{keyword}_{i+1}.jpg"), 'wb') as img_file:
                img_file.write(response.content)
        except Exception as e:
            print(f"An error occurred while downloading image {i+1}: {e}")

if __name__ == "__main__":
    search_keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
    scrape_images(search_keywords)


TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

# Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [54]:
pip install selenium pandas


In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

def scrape_flipkart_smartphones(product_name):
    driver = webdriver.Chrome(executable_path='path/to/chromedriver') 
    driver.get("https://www.flipkart.com/")

    close_popup_button = driver.find_element(By.CSS_SELECTOR, "button._2KpZ6l._2doB4z")
    if close_popup_button.is_displayed():
        close_popup_button.click()

    search_box = driver.find_element(By.NAME, "q")
    search_box.clear()
    search_box.send_keys(product_name)
    search_box.submit()

    driver.implicitly_wait(10)

    details_list = []
    product_cards = driver.find_elements(By.CSS_SELECTOR, "div._1AtVbE")

    for card in product_cards:
        details = extract_product_details(card)
        details_list.append(details)

    columns = ["Brand Name", "Smartphone Name", "Colour", "RAM", "Storage (ROM)", 
               "Primary Camera", "Secondary Camera", "Display Size", 
               "Battery Capacity", "Price", "Product URL"]
    df = pd.DataFrame(details_list, columns=columns)

    df.to_csv("flipkart_smartphones.csv", index=False)

    driver.quit()

def extract_product_details(card):
    details = {}
    details["Brand Name"] = card.find_element(By.CSS_SELECTOR, "div._2WkVRV").text
    details["Smartphone Name"] = card.find_element(By.CSS_SELECTOR, "a._1fQZEK").text
    details["Colour"] = get_text(card, "div._2o7WAb > ul > li:nth-child(4)")
    details["RAM"] = get_text(card, "div._2o7WAb > ul > li:nth-child(1)")
    details["Storage (ROM)"] = get_text(card, "div._2o7WAb > ul > li:nth-child(2)")
    details["Primary Camera"] = get_text(card, "div._2o7WAb > ul > li:nth-child(3)")
    details["Secondary Camera"] = get_text(card, "div._2o7WAb > ul > li:nth-child(5)")
    details["Display Size"] = get_text(card, "div._2o7WAb > ul > li:nth-child(6)")
    details["Battery Capacity"] = get_text(card, "div._2o7WAb > ul > li:nth-child(7)")
    details["Price"] = card.find_element(By.CSS_SELECTOR, "div._30jeq3").text
    details["Product URL"] = card.find_element(By.CSS_SELECTOR, "a._1fQZEK").get_attribute("href")

    return details

def get_text(element, selector):
    try:
        return element.find_element(By.CSS_SELECTOR, selector).text
    except:
        return "-"

if __name__ == "__main__":
    product_name = input("Enter the smartphone you want to search for on Flipkart: ")
    scrape_flipkart_smartphones(product_name)


KeyboardInterrupt: Interrupted by user

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [57]:
pip install geopy

In [62]:
from geopy.geocoders import ArcGIS
nom = ArcGIS()
nom.geocode('shantinagar')

Location(Shantinagar, Bangalor, Karnatak, (12.95756, 77.59693, 0.0))

In [63]:
nom.geocode('modinagar')

Location(Modinagar, Ghaziabad, Uttar Pradesh, (28.83624, 77.57679, 0.0))

In [64]:
nom.geocode('delhi')

Location(Delhi, Connaught Place, New Delhi, Delhi, (28.63409, 77.21693, 0.0))

# 6. Write a program to scrap all the available details of best gaming laptops from digit.in.

In [65]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_digit_gaming_laptops():
    url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        laptops = []

        laptop_cards = soup.find_all("div", class_="TopNumbeHeading")

        for card in laptop_cards:
            details = get_laptop_details(card)
            laptops.append(details)

        columns = ["Name", "Processor", "RAM", "OS", "Storage", "Display", "Price"]
        df = pd.DataFrame(laptops, columns=columns)

        print(df)
        df.to_csv("digit_gaming_laptops.csv", index=False)

    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

def get_laptop_details(card):
    details = {}
    details["Name"] = card.find("div", class_="TopNumbeHeading").text.strip()
    details["Processor"] = card.find("div", class_="prod-spec").text.strip()
    details["RAM"] = card.find("div", class_="Spcs-details").find_all("td")[1].text.strip()
    details["OS"] = card.find("div", class_="Spcs-details").find_all("td")[3].text.strip()
    details["Storage"] = card.find("div", class_="Spcs-details").find_all("td")[5].text.strip()
    details["Display"] = card.find("div", class_="Spcs-details").find_all("td")[7].text.strip()
    details["Price"] = card.find("div", class_="TopPriceBox").text.strip()

    return details

if __name__ == "__main__":
    scrape_digit_gaming_laptops()


Empty DataFrame
Columns: [Name, Processor, RAM, OS, Storage, Display, Price]
Index: []


# Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [67]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_forbes_billionaires():
    url = "https://www.forbes.com/billionaires/"

    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        table = soup.find("table", class_="data")

        if table:
            billionaires = []

            rows = table.find_all("tr")[1:] 

            for row in rows:
                details = get_billionaire_details(row)
                billionaires.append(details)

            columns = ["Rank", "Name", "Net Worth", "Age", "Citizenship", "Source", "Industry"]
            df = pd.DataFrame(billionaires, columns=columns)

            print(df)
            df.to_csv("forbes_billionaires.csv", index=False)

        else:
            print("Table not found on the page.")

    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

def get_billionaire_details(row):
    details = {}
    columns = row.find_all("td")

    details["Rank"] = columns[0].text.strip()
    details["Name"] = columns[1].text.strip()
    details["Net Worth"] = columns[2].text.strip()
    details["Age"] = columns[3].text.strip()
    details["Citizenship"] = columns[4].text.strip()
    details["Source"] = columns[5].text.strip()
    details["Industry"] = columns[6].text.strip()

    return details

if __name__ == "__main__":
    scrape_forbes_billionaires()


Table not found on the page.


# 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video

In [70]:
from selenium import webdriver
import time

# Set up the WebDriver
driver = webdriver.Chrome('path_to_chromedriver')  # Replace with the path to your WebDriver executable

# Open the YouTube video
video_url = 'https://www.youtube.com/watch?v=your_video_id'  # Replace with the URL of the YouTube video
driver.get(video_url)


scroll_pause_time = 2  
scrolls = 10  

for _ in range(scrolls):
  driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
  time.sleep(scroll_pause_time)

comments = driver.find_elements_by_xpath('//yt-formatted-string[@id="content-text"]')
upvotes = driver.find_elements_by_xpath('//span[@id="vote-count-middle"]')
times = driver.find_elements_by_xpath('//a[@class="yt-simple-endpoint style-scope yt-formatted-string"]')

extracted_data = []
for comment, upvote, time in zip(comments, upvotes, times):
  extracted_data.append({
  'comment': comment.text,
  'upvote': upvote.text,
  'time': time.text
  })

driver.quit()
for data in extracted_data:
  print(data)

AttributeError: 'str' object has no attribute 'capabilities'

# Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [71]:
import requests
from bs4 import BeautifulSoup

url = "https://www.hostelworld.com/hostels/London"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
hostels = soup.find_all("div", class_="fabresult")

for hostel in hostels:
    name = hostel.find("h2", class_="fabresult-title").text.strip()
    distance = hostel.find("span", class_="distance").text.strip()
    ratings = hostel.find("div", class_="rating").text.strip()
  
    total_reviews = hostel.find("div", class_="reviews").text.strip()
    overall_reviews = hostel.find("div", class_="overall").text.strip()
    privates_price = hostel.find("div", class_="price-col").find("div", class_="price").text.strip()
    dorms_price = hostel.find("div", class_="price-col").find("div", class_="price").find_next_sibling("div", class_="price").text.strip()
    facilities = hostel.find("div", class_="facilities").text.strip()
    description = hostel.find("div", class_="description").text.strip()
  
  
    print("Hostel Name:", name)
    print("Distance from City Centre:", distance)
    print("Ratings:", ratings)
    print("Total Reviews:", total_reviews)
    print("Overall Reviews:", overall_reviews)
    print("Privates from Price:", privates_price)
    print("Dorms from Price:", dorms_price)
    print("Facilities:", facilities)
    print("Property Description:", description)
    print()